In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rc('font', family='NanumGothic')#차트 레이블에 한글이 깨지는 현상방지

In [ ]:
df=pd.read_csv('users.csv')

In [ ]:
df.head()

In [ ]:
data=df[['age','gender','skin_type']]

In [ ]:
plt.subplots(figsize=(20,5))
sns.distplot(data.age, hist=False, label='1', kde=True, rug=True)#나이 분포 차트를 그려줍니다

In [ ]:
sns.countplot(data=data, x='gender', hue='gender') #성별 개수 확인

In [ ]:
sns.countplot(data=data, x='skin_type', hue='skin_type')# 피부타입별 변수의 개수 확인

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
traceW = go.Box(x = data['age'].values,
                name="Width",
                marker=dict(color='rgba(238,23,11,0.5)',
                            line=dict(color='red', width=1.2)),
                orientation='h') #plotly패키지를 이용해 상자그림을 그려서 나이 변수의 이상 값 확인 

                
box = [traceW]

layout = dict(title = 'Width & Heights of images',
              xaxis = dict(title = 'Size', showticklabels=True),
              yaxis = dict(title = 'Image dimmension'),
              hovermode = 'closest')

fig = dict(data=box, layout=layout)

plot(fig, filename='width-height.html') #width-height.html라는 파일로 차트 결과가 나옵니다.

In [ ]:
data=data.loc[(data['age']<36)&(data['age']>10)] #확인한 이상값을 제거해줍니다.

In [ ]:
plt.subplots(figsize=(20,5)) #이상 값들을 제거한 나이분포 그래프를 다시 그려봅니다.
sns.distplot(data.age, hist=False, label='1', kde=True, rug=True) 

In [ ]:
from scipy.stats import skew #나이 분포의 왜도 확인 0에 가까울수록 좌우 대칭에 가깝습니다,
skew(data['age'])

In [ ]:
#범주형 변수들을 더미변수화 시켜줍니다.
data = data.merge(pd.get_dummies(data.skin_type, prefix='skin_type'), left_index=True, right_index=True)
data = data.merge(pd.get_dummies(data.gender, prefix='gender'), left_index=True, right_index=True)

In [ ]:
#더미변수 만들때 사용한 원래 변수 제거
del data['skin_type']
del data['gender']

In [ ]:
#변수 스케일 조정을 하게되면 columns값이 사라지므로 미리 확인해 봅니다.
data.columns

In [ ]:
#군집분석을 위해 standard scaler로 변수들의 스케일 조정
from sklearn.preprocessing import StandardScaler
df2 = StandardScaler().fit_transform(data)
df2=pd.DataFrame(df2)

In [ ]:
df2.columns=['age', 'skin_type_건성', 'skin_type_민감성', 'skin_type_복합성', 'skin_type_중성',
       'skin_type_지성', 'gender_1', 'gender_2']

In [ ]:
df2.head()

군집의 수를 정하기 위해 실루엣 계수 사용
실루엣 계수는 -1부터 1 사이의 값을 가지고 샘플의 실루엣 계수가 1이면 그 샘플은 속한 크러스터와 매우 유사하고 다른 클러스터와 다르다는것을 의미 
군집화 전체를 평가하기위해 사용하는 실루엣 계수는 각 샘플의 실루엣 계수의 평균을 사용

In [ ]:
from sklearn.cluster import KMeans #군집분석 방법은 kmeans 사용
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score

for n_clusters in range(2,10):

    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(df2)

    silhouette_avg = silhouette_score(df2, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)


엘보방식은 급내제곱합 이용 클러스터들의 급내제곱합의 총합이 적을수록 좋은 군집화 사이킷런의 kmeans.fit함수는 inertia_라는 이름으로 클러스터의 급내제곱합의 총합을 계산하므로 쉽게 구할수 있음

In [ ]:
# 클러스터 수를 키로 하고 inertia를 값으로 하는 딕셔너리
ssw_dic={}

for k in range(1,10):
    km = KMeans(n_clusters=k).fit(df2)
    ssw_dic[k]= km.inertia_
    
plot_data_x=list(ssw_dic.keys())
plot_data_y=list(ssw_dic.values())
plt.xlabel('# of clusters')
plt.ylabel('within ss')
plt.plot(plot_data_x,plot_data_y,linestyle='-',marker='o')
plt.title('The Elbow Method showing the optimal k')
plt.show()

군집의 수를 6개로 하겠습니다.

In [ ]:
kmeans = KMeans(n_clusters=6)
predict=kmeans.fit(df2)

In [ ]:
out=pd.DataFrame(kmeans.labels_)
out.columns=['predict']
data2=pd.concat([df2,out],1)

In [ ]:
data2.head()

In [ ]:
data2.nunique()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# scatter plot
fig = plt.figure( figsize=(6,6))
ax = Axes3D(fig, rect=[0, 1, 1, 1], elev=48, azim=134)
ax.scatter(data2['age'],data2['skin_type_민감성'],data2['gender_2'],c=data2['predict'],alpha=0.5)
ax.set_xlabel('age')
ax.set_ylabel('skin_type_민감성')
ax.set_zlabel('gender_2')
plt.show()

In [ ]:
data2.isna().sum()

In [ ]:
X = data2.iloc[:, data2.columns != 'predict']
y = data2.iloc[:, data2.columns == 'predict']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 10)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [ ]:
kfold = StratifiedKFold(n_splits=10)

In [ ]:
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")


In [ ]:
gbc = GradientBoostingClassifier()
gbc = gbc.fit(X_train, y_train)

In [ ]:
gbc_scores = cross_val_score(gbc, X_train, y_train,cv=10)
print('Scores =', gbc_scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(gbc_scores), np.std(gbc_scores)))

In [ ]:
from sklearn.metrics import classification_report
y_pred=gbc.predict(X_test)
# classification matrix
print('\nClassification metrics')
print(classification_report(y_true=y_test, y_pred=y_pred))